In [ ]:
import pyspark

In [ ]:
! pip install numpy pandas matplotlib seaborn scikit-learn

In [13]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count, upper, lit

In [3]:
type(pd.read_csv('train.csv'))

/var/folders/vq/9ksqctns5zl1tj5792ttp1qm0000gn/T/ipykernel_47101/1081934449.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  type(pd.read_csv('train.csv'))


pandas.core.frame.DataFrame

In [5]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/openjdk-11.jdk/Contents/Home"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [6]:
print(os.environ.get("JAVA_HOME"))

/Library/Java/JavaVirtualMachines/openjdk-11.jdk/Contents/Home


In [7]:
spark=SparkSession.builder.appName("DemoApp").getOrCreate()
df_spark=spark.read.csv('train.csv',header=True,inferSchema=True)
df_spark.show(5)

25/11/21 18:41:54 WARN Utils: Your hostname, Shithils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.226 instead (on interface en0)
25/11/21 18:41:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/21 18:41:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/21 18:41:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
only showing top 5 rows



In [8]:
df_spark.printSchema()

root
 |-- Store: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Sales: integer (nullable = true)
 |-- Customers: integer (nullable = true)
 |-- Open: integer (nullable = true)
 |-- Promo: integer (nullable = true)
 |-- StateHoliday: string (nullable = true)
 |-- SchoolHoliday: integer (nullable = true)



In [10]:
df_spark.select('Date','Sales').show(5)

+----------+-----+
|      Date|Sales|
+----------+-----+
|2015-07-31| 5263|
|2015-07-31| 6064|
|2015-07-31| 8314|
|2015-07-31|13995|
|2015-07-31| 4822|
+----------+-----+
only showing top 5 rows



In [14]:
df_spark.filter(col('Sales')>5000).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|
|    6|        5|2015-07-31| 5651|      589|   1|    1|           0|            1|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
only showing top 5 rows



In [15]:
df_spark.withColumn('Sales_Upper', upper(col('Sales'))).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Sales_Upper|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|       5263|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|       6064|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|       8314|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|      13995|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|       4822|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------+
only showing top 5 rows



In [20]:
df_spark.groupBy('Store').agg(avg('Sales').alias('Avg_Sales'),sum('Sales').alias('Total_Sales'),count('Sales').alias('Count_Sales')).show(5)

+-----+------------------+-----------+-----------+
|Store|         Avg_Sales|Total_Sales|Count_Sales|
+-----+------------------+-----------+-----------+
|  148| 7189.967091295117|    6772949|        942|
|  463| 5040.073248407643|    4747749|        942|
|  471|  5834.12401055409|    4422266|        758|
|  496| 6432.545647558386|    6059458|        942|
|  833|4557.6878980891715|    4293342|        942|
+-----+------------------+-----------+-----------+
only showing top 5 rows



In [21]:
df_spark.orderBy(col('Sales').desc()).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|  909|        1|2015-06-22|41551|     1721|   1|    0|           0|            0|
|  262|        5|2015-04-03|38722|     5132|   1|    1|           b|            0|
|  262|        5|2015-05-01|38484|     5458|   1|    1|           a|            0|
|  262|        4|2015-05-14|38367|     5192|   1|    0|           a|            1|
|   57|        1|2014-06-16|38037|     1970|   1|    1|           0|            0|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
only showing top 5 rows



In [ ]:
df_spark.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
only showing top 5 rows



In [29]:
df_spark.select('Store','Sales','Customers').where((col('Sales')>4000) & (col('Customers')<500)).show()

+-----+-----+---------+
|Store|Sales|Customers|
+-----+-----+---------+
|   41| 6938|      494|
|   45| 6301|      442|
|   50| 5729|      441|
|  103| 5425|      453|
|  182| 6755|      491|
|  186| 4957|      447|
|  210| 4275|      409|
|  212| 6049|      471|
|  232| 5738|      469|
|  240| 5195|      474|
|  242| 5056|      451|
|  254| 4239|      315|
|  263| 5216|      440|
|  266| 4779|      495|
|  286| 5093|      438|
|  288| 6886|      480|
|  302| 5356|      393|
|  330| 5509|      489|
|  337| 7345|      493|
|  373| 6621|      405|
+-----+-----+---------+
only showing top 20 rows



In [30]:
df_spark.groupBy('Store').agg(avg('Customers').alias('Avg_Customers'),sum('Customers').alias('Total_Customers')).show(5)

+-----+------------------+---------------+
|Store|     Avg_Customers|Total_Customers|
+-----+------------------+---------------+
|  148| 682.0276008492569|         642470|
|  463| 801.6815286624204|         755184|
|  471|484.31002638522426|         367107|
|  496|  690.995753715499|         650918|
|  833| 418.5944798301486|         394316|
+-----+------------------+---------------+
only showing top 5 rows



Transformations

In [31]:
new_df=df_spark.withColumn("Customers_per_100_Sales",
    (col("Customers") / col("Sales")) * 100)

In [34]:
new_df.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Customers_per_100_Sales|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|     10.545316359490785|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|     10.306728232189974|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|      9.874909790714458|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|     10.703822793854949|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|     11.592700124429697|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
only showing top 5 rows



In [38]:
analysis_df=df_spark.withColumn('Customers_per_Store_day', col('Customers')/lit(7))
analysis_df.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Customers_per_Store_day|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|      79.28571428571429|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|      89.28571428571429|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|     117.28571428571429|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|                  214.0|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|      79.85714285714286|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
only showing top 5 rows



In [48]:
analysis_df.where((col('Sales')>8000) & (col('Customers')<500)).show()

+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Customers_per_Store_day|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+
|  612|        5|2015-07-31| 8161|      493|   1|    1|           0|            0|      70.42857142857143|
|  455|        2|2015-07-28| 9293|      492|   1|    1|           0|            0|      70.28571428571429|
|  866|        2|2015-07-28| 8020|      494|   1|    1|           0|            1|      70.57142857142857|
|  612|        5|2015-07-17| 8344|      458|   1|    1|           0|            0|      65.42857142857143|
|  158|        4|2015-07-16| 8328|      494|   1|    1|           0|            0|      70.57142857142857|
|   13|        3|2015-07-15|10334|      415|   1|    1|           0|            0|     59.285714285714285|
|  455|        3|2015-07-15| 8006|   

In [51]:
from pyspark.sql.functions import log
# The log1p transformation is useful for transforming data that spans several orders of magnitude.
analysis_df.withColumn('Sales_Log1p', log(lit(1)+col('Sales'))).where(col('Sales')>1000).orderBy(col('Sales').desc()).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Customers_per_Store_day|       Sales_Log1p|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------------+------------------+
|  909|        1|2015-06-22|41551|     1721|   1|    0|           0|            0|     245.85714285714286| 10.63470093390253|
|  262|        5|2015-04-03|38722|     5132|   1|    1|           b|            0|      733.1428571428571|10.564189017727987|
|  262|        5|2015-05-01|38484|     5458|   1|    1|           a|            0|      779.7142857142857|10.558023833968418|
|  262|        4|2015-05-14|38367|     5192|   1|    0|           a|            1|      741.7142857142857|10.554979057827241|
|   57|        1|2014-06-16|38037|     1970|   1|    1|           0|            0|     281.42857142857144|10.546340939

25/11/21 22:05:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1037670 ms exceeds timeout 120000 ms
25/11/21 22:05:32 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/21 22:22:41 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 